# Example of a Camera Display to HTML5 via MPLD3

if you don't already have mpld3, install it via the following command:

In [2]:
! pip install --user mpld3

    100% |████████████████████████████████| 1.1MB 378kB/s 
  Running setup.py install for mpld3


Otherwise, start here:

In [14]:
%matplotlib inline
import mpld3

In [15]:
# %load camera_display.py


"""
Example of drawing a Camera using a mock shower image
"""

import matplotlib.pylab as plt
from ctapipe import io, visualization
from ctapipe.reco import mock
from ctapipe.reco import hillas_parameters_2 as hillas_parameters
import numpy as np


def draw_neighbors(geom, pixel_index, color='r', **kwargs):
    """ draw lines between a pixel and its neighbors"""
    neigh = geom.neighbors[pixel_index]  # neighbor indices (not pixel ids)
    x, y = geom.pix_x[pixel_index].value, geom.pix_y[pixel_index].value
    for nn in neigh:
        nx, ny = geom.pix_x[nn].value, geom.pix_y[nn].value
        plt.plot([x, nx], [y, ny], color=color, **kwargs)


# load the camera
geom = io.get_camera_geometry("hess", 1)
disp = visualization.CameraDisplay(geom)

# create a fake camera image to display:
model = mock.generate_2d_shower_model(centroid=(0.2, 0.0),
                                      width=0.01,
                                      length=0.1,
                                      psi=np.radians(35))

image, sig, bg = mock.make_mock_shower_image(geom, model.pdf, intensity=50,
                                             nsb_level_pe=1000)

# apply really stupid image cleaning (single threshold):
clean = image.copy()
clean[image <= 3.0 * image.mean()] = 0.0

# calculate image parameters
hillas = hillas_parameters(geom.pix_x.value, geom.pix_y.value, clean)
print(hillas)

# show the camera image and overlay Hillas ellipse
disp.set_image(image)
disp.overlay_moments(hillas, edgecolor='black', linewidth=3,zorder=10)

# draw the neighbors of pixel 100 in red, and the
# neighbor-neighbors in green
for ii in geom.neighbors[130]:
    draw_neighbors(geom, ii, color='green')
draw_neighbors(geom, 130, color='red',lw=2)

mpld3.display()

MomentParameters(size=22676.125, cen_x=0.18931349770330591, cen_y=0.0019718880885605467, length=0.31681940045765189, width=0.13533022097484754, r=0.18932376701115319, phi=0.010415616451880712, psi=0.63807824801928925, miss=0.15323910158396431)


The plot above is in HTML5 with javascript, so you can zoom interactively. The conversion was done automatically by _mpld3_ 